# Modelo de Aprendizaje Automático usando Similitud del Coseno

Para construir un modelo de recomendación de juegos utilizando similitud del coseno, es fundamental determinar qué características utilizar para calcular esa similitud. En este caso, luego de realizar la debida diligencia asociada al EDA del archivo de datos que será utilizado, se considera que las columnas que contienen información relevante para medir similitudes entre juegos son: 'genres', 'specs', 'developer' y 'id'. La columna 'id' podría ser útil para identificar de manera única cada juego y también para evitar que se recomiende el mismo juego que se está buscando.

## Proceso o Flujo de trabajo 

El flujo de trabajo para el desarrollo del Modelo de Aprendizaje Automático en la modalidad de sistema de recomendación item-item se presenta a continuación. Este proceso define un sistema básico de recomendación de juegos usando similitud de coseno con TF-IDF (Term Frequency-Inverse Document Frequency) en texto. La lógica es combinar varias columnas relevantes en una sola columna de "características combinadas" y luego calcular la similitud de coseno entre los juegos basándose en esas características:

1. Preparación de datos: Se combina información relevante de las columnas ('genres', 'specs', 'developer', 'id') en una columna llamada 'combined_features'. 
IMPORTANTE: En el contexto de un modelo de recomendación basado en similitud del coseno, no es común dividir los datos en conjuntos de entrenamiento y prueba, como se hace típicamente en el aprendizaje supervisado. Esto se debe a que el modelo de recomendación no está prediciendo una etiqueta o variable objetivo específica.  

2. Vectorización TF-IDF: Se inicializa un vectorizador TF-IDF para convertir el texto en una representación numérica.
Se aplica el vectorizador a la columna 'combined_features' para crear una matriz TF-IDF que representa las características de los juegos.

3. Cálculo de similitud del coseno: Se calcula la similitud del coseno entre los juegos basándose en la matriz TF-IDF generada.

4. Función de recomendación: Se define una función (recomendacion_juego) que toma el id de un juego como entrada y devuelve los 5 juegos más similares basados en la similitud del coseno.

5. Obtener recomendaciones: Se llama a la función recomendacion_juego con el id de un juego específico para obtener recomendaciones de 5 juegos similares.


## Implementación
Se utilizará la librería de SciKit-learn de python para importar las clases TfidfVectorizer y cosine_similarity.

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Carga el archivo Parquet 'Clean_output_steam_games.parquet' en DataFrame 'df2'
df2 = pd.read_parquet('../Datasets/Clean_Parquet_Data_Steam/Clean_output_steam_games.parquet')

# 1. Preparación de datos:
# Combina columnas relevantes en una columna 'combined_features'
df2['combined_features'] = df2['genres'] + ' ' + df2['specs'] + ' ' + df2['developer'] + ' ' + df2['id']

# 2. Vectorización TF-IDF:
# Inicializa el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()
# Transforma la columna 'combined_features' en una matriz TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(df2['combined_features'])

# 3. Cálculo de similitud del coseno:
# Calcula la similitud del coseno entre juegos
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [21]:
# 4. Función de recomendación:
# Función para obtener recomendaciones
def recomendacion_juego(id):
    # Obtiene el índice del id de juego proporcionado
    idx = df2[df2['id'] == id].index[0]
    
    # Obtiene las puntuaciones de similitud del juego en cuestión con otros juegos
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordena los juegos según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtiene los índices de los juegos más similares (excluyendo el propio juego)
    similar_games_indices = [i[0] for i in sim_scores[1:6]]  # Recomendar los 5 juegos más similares
    
    # Devuelve los nombres de los juegos recomendados
    #return df2['app_name'].iloc[similar_games_indices]

    # Crea una lista de diccionarios con 'id' como clave y 'app_name' como valor
    recommended_games = [{df2['id'].iloc[ind] : df2['app_name'].iloc[ind]} for ind in similar_games_indices]

    return recommended_games

# 5. Obtener recomendaciones:
# Obtiene recomendaciones para un juego específico (cambia 'id' por el del juego que desees)
recommended_games = recomendacion_juego('670290')
recommended_games

[{'504350': 'RoShamBo: Starter Fighter Pack'},
 {'321400': 'Supraball'},
 {'393930': 'RoShamBo'},
 {'583740': 'AVATARIKA'},
 {'522660': 'Snooker-online multiplayer snooker game!'}]